# What's the effective vocabulary size with inversions?

- min : 3  (1, b3, 5)
- maj : 3  (1, 3, 5)
- dim : 3  (1, b3, b5)
- aug : 3  (1, 3, #5)
- min7 : 4 (1, b3, 5, b7)
- maj7 : 4 (1, 3, 5, 7)
- dom7 : 4 (1, 3, 5, b7)
- dim7 : 4 (1, b3, b5, b7)
- hdim7 : 4 (1, b3, b5, 7)
- minmaj7 : 4 (1, b3, 5, 7)
- min6 : 4 (1, b3, 5, 6)
- maj6 : 4 (1, 3, 5, 6)
- sus2 : 3 (1, 2, 5)
- sus4 : 3 (1, 4, 5)
- X : 1
- N : 1

- Without inversions: $12 * (4 + 6 + 2 + 2) + 2 = 170$
- With inversions: $12 * (3 * 4 + 4 * 6 + 4 * 2 + 3 * 2) + 2 = 602$

In [33]:
import os
import jams
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from joblib import Parallel, delayed

In [6]:
def getid(fname):
    return os.path.splitext(os.path.basename(fname))[0]

In [85]:
def compare(ref_file, est_file, corpus):
    
    ref_jam = jams.load(ref_file, validate=False)
    est_jam = jams.load(est_file, validate=False)
    
    ref_ann = ref_jam.search(corpus=corpus)[0]
    est_ann = est_jam.annotations[0]
    est_ann = est_ann.trim(0, ref_ann.data[-1].time + ref_ann.data[-1].duration)
    
    results = jams.eval.chord(ref_ann, est_ann)
    
    return {getid(est_file): dict(results)}

In [72]:
def eval_model(ids, corpus, path):
    
    results = {}
    for tid in tqdm(ids):
        try:
            ref_file = os.path.join(REF_PATH, os.path.extsep.join([tid, 'jams']))
            est_file = os.path.join(path, os.path.extsep.join([tid, 'jams']))
            results.update(compare(ref_file, est_file, corpus))
        except ValueError as exc:
            print('Failure at track {}'.format(est_file))
            print(exc)
            
    return pd.DataFrame.from_dict(results, orient='index').sort_index(axis=1)

In [36]:
REF_PATH = '/home/bmcfee/data/eric_chords/references_v2/'

In [37]:
# MARL-mir analysis
marl_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/marl-chords.csv', squeeze=True, header=None)
marl_corp = 'MARL-Chords'

In [51]:
marl_cr2si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

A Jupyter Widget

In [86]:
marl_cr2s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [52]:
marl_cr2 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

A Jupyter Widget

In [53]:
marl_cr1si = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

A Jupyter Widget

In [57]:
marl_cr1s = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

A Jupyter Widget

In [54]:
marl_cr1 = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

A Jupyter Widget

In [61]:
marl_chordino = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

A Jupyter Widget

In [66]:
marl_dnn = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

A Jupyter Widget

In [92]:
marl_khmm = eval_model(marl_ids, marl_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


In [93]:
marl_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_chordino.json')
marl_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_dnn.json')
marl_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_khmm.json')

marl_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1.json')
marl_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1s.json')
marl_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr1si.json')

marl_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2.json')
marl_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2s.json')
marl_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/marl_cr2si.json')


In [18]:
# Isophonics
iso_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/isophonics.csv', squeeze=True, header=None)
iso_corp = 'Isophonics'

In [97]:
iso_cr2si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

iso_cr2s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

iso_cr2 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

iso_cr1si = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

iso_cr1s = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

iso_cr1 = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

iso_chordino = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

iso_dnn = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

iso_khmm = eval_model(iso_ids, iso_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [98]:
iso_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_chordino.json')
iso_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_dnn.json')
iso_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_khmm.json')

iso_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1.json')
iso_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1s.json')
iso_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr1si.json')

iso_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2.json')
iso_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2s.json')
iso_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/iso_cr2si.json')


In [100]:
# Billboard analysis
bb_ids = pd.read_csv('/home/bmcfee/git/ace-lessons/data/chord_dataset/billboard.csv', squeeze=True, header=None)
bb_corp = 'Billboard-Chords'

In [101]:
bb_cr2si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions-inv/')

bb_cr2s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_struct_aug/predictions/')

bb_cr2 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_deep_aug/predictions/')

bb_cr1si = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions-inv/')

bb_cr1s = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_struct_aug/predictions/')

bb_cr1 = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/model_aug/predictions/')

bb_chordino = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/chordino/')

bb_dnn = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_dnn/predictions/')

bb_khmm = eval_model(bb_ids, bb_corp, '/home/bmcfee/git/ismir2017_chords/data/ejh2015_khmm/predictions/')

A Jupyter Widget

/home/bmcfee/git/jams/jams/core.py:876: UserWarning: Annotation.duration is not defined, cannot check for temporal intersection, assuming the annotation is valid between start_time and end_time.
  "Annotation.duration is not defined, cannot check "


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [102]:
bb_chordino.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_chordino.json')
bb_dnn.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_dnn.json')
bb_khmm.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_khmm.json')

bb_cr1.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1.json')
bb_cr1s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1s.json')
bb_cr1si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr1si.json')

bb_cr2.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2.json')
bb_cr2s.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2s.json')
bb_cr2si.to_json('/home/bmcfee/git/ismir2017_chords/data/bb_cr2si.json')


In [107]:
bb_cr2si.describe()

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
count,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000
mean,0.797925,0.777841,0.802754,0.818529,0.671596,0.658024,0.617955,0.605756,0.780992,0.759269,0.746970,0.728710
std,0.172268,0.177999,0.152424,0.140904,0.205586,0.206759,0.220739,0.221504,0.172217,0.178817,0.197946,0.202027
min,0.025418,0.025418,0.025610,0.026430,0.018132,0.018132,0.006793,0.006793,0.023833,0.023833,0.006793,0.006793
25%,0.744448,0.709386,0.743646,0.760695,0.565208,0.552622,0.481067,0.473811,0.711529,0.680954,0.664886,0.633912
50%,0.848406,0.826679,0.842717,0.852393,0.719829,0.702318,0.665065,0.646081,0.827919,0.803653,0.805052,0.779911
75%,0.915057,0.905798,0.909802,0.916492,0.821496,0.809925,0.782425,0.772214,0.905982,0.894824,0.893645,0.882690
max,0.999719,0.999719,0.999719,0.999117,0.999719,0.999719,0.984466,0.984466,0.989286,0.989286,0.989286,0.989286


In [109]:
bb_khmm.describe()

,majmin,majmin_inv,mirex,root,sevenths,sevenths_inv,tetrads,tetrads_inv,thirds,thirds_inv,triads,triads_inv
count,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000
mean,0.762479,0.739064,0.773405,0.796638,0.635168,0.616872,0.589807,0.573386,0.764016,0.739879,0.721274,0.700102
std,0.172426,0.179143,0.165483,0.157450,0.196366,0.199170,0.211100,0.213154,0.170281,0.177656,0.197513,0.202170
min,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911,0.004911
25%,0.698624,0.661413,0.713129,0.742862,0.539728,0.514947,0.469591,0.444049,0.696056,0.661316,0.643759,0.606200
50%,0.806293,0.777561,0.816213,0.834545,0.659567,0.639930,0.614937,0.599487,0.807019,0.780961,0.776540,0.746125
75%,0.882482,0.866837,0.885568,0.897166,0.777148,0.756984,0.743446,0.732469,0.881147,0.866139,0.864970,0.846247
max,0.985987,0.984107,0.986186,0.988760,0.984107,0.984107,0.968284,0.968284,0.986365,0.980920,0.986186,0.980920
